## 0-1 Knapsack Problem

### Instance Formulation

\begin{align*}
    & \max_x 17x_1+10x_2 + 25x_3 + 17x_4 \\
    & \text{    s.t.: } 5x_1 + 3x_2 + 8x_3 + 7x_4 \leq 12 \\
    & \ \ \ \ \ \ \ x_i \in \{0, 1 \}, i = 1,\cdots, 4.
\end{align*}

In [6]:
using JuMP
using Cbc
using GLPK

### Parameter matrices

In [73]:
# c as a column vector
# c = [17 10 25 17] defines a 1×4 Matrix, 
# which is not supported by JuMP when forming objective formula c*x
c = [17; 10; 25; 17]
A1 = [5 3 8 7;]
b = [12;]
;

### Model setup

In [78]:
function matrix_formulation(optimizer, A, b, c)
    # A: non-binary constraint coefficient matrix
    # b: non-binary constraint constant (column) vector
    # c: cost/profit (column) vector  

    model = Model(optimizer)
    
    # Type I definition
    # Set of variables x
    J = 1:size(c)[1]
    @variable(model, x[J], Bin)
    @objective(model, Max, c'*x.data)
    @constraint(model, A*x.data .<= b)

    # # Type II definition
    # # Number of variables x
    # nJ = size(c)[1]
    # @variable(model, x[1:nJ], Bin)
    # @objective(model, Max, c'*x)
    # @constraint(model, A*x .<= b)

    # set_silent(model)
    optimize!(model)
    
    status = termination_status(model)
    println(status)

    return model

end

function equation_formulation(optimizer, A, b, c)
    model = Model(optimizer)
    # Set of variables x
    J = 1:size(c)[1]

    # Set of the row of non-binary constraint coefficient matrix
    I = 1:size(b)[1]

    @variable(model, x[J], Bin)
    @constraint(model, Coeff[i in I], sum(A[i,j]*x[j] for j in J) .<= b)
    @objective(model, Max, sum(c[j]*x[j] for j in J))
    
    # set_silent(model)
    optimize!(model)

    status = termination_status(model)
    println(status)

    return model
end

model = equation_formulation(Cbc.Optimizer, A1, b, c)



Welcome to the CBC MILP Solver 
Version: 2.10.5 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 39.5 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0004I processed model has 1 rows, 4 columns (4 integer (4 of which binary)) and 4 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 1 integers unsatisfied sum - 0.444444
Cbc0038I Solution found of -27
Cbc0038I Before mini branch and bound, 3 integers at bound fixed and 0 continuous
Cbc0038I Full problem 1 rows 4 columns, reduced to 0 rows 0 columns
Cbc0038I Mini branch and bound did not improve solution (0.00 seconds)
Cbc0038I Round again with cutoff of -29.011
Cbc0038I Pass   1: suminf.    0.08044 (1) obj. -29.011 iterations 1
Cbc0038I Pass   2: suminf.    0.44444 (1) obj. -38.1111 iterations 1
Cbc0038I Pass   3: suminf.    0.08044 (1) obj. -29.011 iterations 1
Cbc0038I Pass   4: suminf.   

A JuMP Model
Maximization problem with:
Variables: 4
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
`VariableRef`-in-`MathOptInterface.ZeroOne`: 4 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)
Names registered in the model: Coeff, x

### Compare defining x with different set types

In [77]:
J = 1:4

dummy_model = Model()

@variable(dummy_model, z[J], Bin)
@variable(dummy_model, y[1:4], Bin)

@show z 
@show z.data
@show y;

z = 1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, 1:4
And data, a 4-element Vector{VariableRef}:
 z[1]
 z[2]
 z[3]
 z[4]
z.data = VariableRef[z[1], z[2], z[3], z[4]]
y = VariableRef[y[1], y[2], y[3], y[4]]


┌ Warning: The model has been modified since the last call to `optimize!` (or `optimize!` has not been called yet). If you are iteratively querying solution information and modifying a model, query all the results first, then modify the model.
└ @ JuMP C:\Users\jhjiang\.julia\packages\JuMP\R2Knd\src\JuMP.jl:1146


### Print solution

In [79]:
println("Optimal value $(objective_value(model))")

# Compare the difference
println("with x = $(value.(model[:x].data))")
println("with x = $(value.(model[:x]))")

Optimal value 35.0
with x = [0.0, 1.0, 1.0, 0.0]
with x = 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:4
And data, a 4-element Vector{Float64}:
 0.0
 1.0
 1.0
 0.0
